In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install implicit

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.9/8.9 MB 17.7 MB/s eta 0:00:00


In [ ]:
!pip install polars

In [ ]:
import random
from collections import defaultdict, Counter
from typing import List, Dict, Union
import pickle

import numpy as np
import implicit
from scipy.sparse import lil_matrix
from tqdm import tqdm
import pandas as pd
import polars as pl

In [ ]:
TOP_N = 50
LOCALE = "DE"
VER = "01"
DIR = "/content/drive/MyDrive/kddcup2023/"

# parameters for IMF
FACTORS = 50
LAMBDA = 0.01
ITERATIONS = 500
SEED = 42

In [ ]:
def preprocess(df:pl.DataFrame) -> pl.DataFrame:
    df = df.explode(["prev_items"])
    df = df.with_columns(
        df.select(pl.col("session_id").cumcount().over("session_id").alias("sequence_num"))
    )
    return df

In [ ]:
def train_imf_and_generate_candidates(df:pd.DataFrame):
    # generate sparse matrix
    unique_user_ids = sorted(df["session_id"].unique())
    unique_item_ids = sorted(df["prev_items"].unique())
    user_id2index = dict(zip(unique_user_ids, range(len(unique_user_ids))))
    item_id2index = dict(zip(unique_item_ids, range(len(unique_item_ids))))

    matrix = lil_matrix((len(unique_user_ids), len(unique_item_ids)))
    for _, row in tqdm(df.iterrows()):
        user_index = user_id2index[row["session_id"]]
        item_index = item_id2index[row["prev_items"]]
        matrix[user_index, item_index] = 1.0

    # convert LIL to CSR
    matrix = matrix.tocsr()

    # model initialization
    model = implicit.bpr.BayesianPersonalizedRanking(
        factors = FACTORS,
        regularization = LAMBDA,
        iterations = ITERATIONS,
        random_state = SEED,
    )

    # train model
    model.fit(matrix)

    # generate candidate
    users = []
    candidates = []
    imf_scores = []

    for user_id, user_index in tqdm(user_id2index.items()):
        item_indexes, scores = model.recommend(user_index, matrix[user_index], N=TOP_N, filter_already_liked_items=True)
        for item_index, score in zip(item_indexes, scores):
            users.append(user_id)
            candidates.append(unique_item_ids[item_index])
            imf_scores.append(score)

    candidates = pd.DataFrame({
        "session_id": users,
        "candidate_item": candidates,
        "bpr_score": imf_scores,
    })

    candidates = pl.from_pandas(candidates)
    candidates = candidates \
    .sort(["session_id", "bpr_score"], descending=[False, True]) \
    .with_columns(pl.col("bpr_score").rank(descending=True, method="min").over("session_id").alias("bpr_rank"))
    candidates = candidates.to_pandas()

    return model, user_id2index, item_id2index, candidates

## For local train/eval

In [ ]:
train = pl.read_parquet("/content/drive/MyDrive/kddcup2023/data/preprocessed/task1/train_task1.parquet")
test = pl.read_parquet("/content/drive/MyDrive/kddcup2023/data/preprocessed/task1/test_task1_phase2.parquet")

train = preprocess(train)
test = preprocess(test)
session_df = pl.concat([
    train["prev_items", "locale", "session_id", "sequence_num"],
    test["prev_items", "locale", "session_id", "sequence_num"],
])
session_df = session_df.to_pandas()
session_df = session_df[session_df["locale"] == LOCALE]

In [ ]:
session_df.head(10)


,prev_items,locale,session_id,sequence_num
0,B09W9FND7K,DE,train_0,0
1,B09JSPLN1M,DE,train_0,1
2,B076THCGSG,DE,train_1,0
3,B007MO8IME,DE,train_1,1
4,B08MF65MLV,DE,train_1,2
5,B001B4TKA0,DE,train_1,3
6,B0B1LGXWDS,DE,train_2,0
7,B00AZYORS2,DE,train_2,1
8,B0B1LGXWDS,DE,train_2,2
9,B00AZYORS2,DE,train_2,3


In [ ]:
%%time
model, user_id2index, item_id2index, candidates = train_imf_and_generate_candidates(session_df)

5284516it [04:40, 18807.49it/s]


  0%|          | 0/500 [00:00<?, ?it/s]

100%|██████████| 1215984/1215984 [2:07:55<00:00, 158.42it/s]


CPU times: user 10h 22min 13s, sys: 6h 13min 53s, total: 16h 36min 7s
Wall time: 2h 17min 50s


In [ ]:
# save models
model_name = f"bpr_{VER}_{LOCALE}_model_for_train_or_eval.npz"
user_ids_name = f"bpr_{VER}_{LOCALE}_user_id2index_for_train_or_eval.pickle"
item_ids_name = f"bpr_{VER}_{LOCALE}_item_id2index_for_train_or_eval.pickle"
model.save(DIR + "models/task1/" + model_name)
with open(DIR + "models/task1/" + user_ids_name, "wb") as f:
    pickle.dump(user_id2index, f)
with open(DIR + "models/task1/" + item_ids_name, "wb") as f:
    pickle.dump(item_id2index, f)

# save candidates
file_name = f"bpr_{VER}_{LOCALE}_for_train_or_eval.parquet"
candidates.to_parquet(DIR + "data/interim/candidates/task1/" + file_name)

In [ ]:
candidates.head()

,session_id,candidate_item,bpr_score,bpr_rank
0,test_phase2_0,B0B3HMH1JP,4.126701,1
1,test_phase2_0,B0BB3CG33L,4.053397,2
2,test_phase2_0,B06VTRJGCP,4.001684,3
3,test_phase2_0,B09C8Y8S4Z,3.952890,4
4,test_phase2_0,B08GG8QSQH,3.951476,5


# For test inference

In [ ]:
train = pl.read_parquet("/content/drive/MyDrive/kddcup2023/data/preprocessed/task1/train_task1.parquet")
test = pl.read_parquet("/content/drive/MyDrive/kddcup2023/data/preprocessed/task1/test_task1_phase2.parquet")

In [ ]:
# trainのnext_itemをprev_itemsにappendする
prev_items_list = train["prev_items"].to_list()
next_item_list = train["next_item"].to_list()
prev_items_list_updated = []
for prev_items, next_item in zip(prev_items_list, next_item_list):
    prev_items.append(next_item)
    prev_items_list_updated.append(prev_items)

train = train.with_columns(
    pl.Series(name="prev_items", values=prev_items_list_updated)
)

In [ ]:
train = preprocess(train)
test = preprocess(test)
session_df = pl.concat([
    train["prev_items", "locale", "session_id", "sequence_num"],
    test["prev_items", "locale", "session_id", "sequence_num"],
])
session_df = session_df.to_pandas()
session_df = session_df[session_df["locale"] == LOCALE]

In [ ]:
%%time
model, user_id2index, item_id2index, candidates = train_imf_and_generate_candidates(session_df)

6395932it [05:34, 19131.17it/s]


  0%|          | 0/500 [00:00<?, ?it/s]

100%|██████████| 1215984/1215984 [2:15:16<00:00, 149.81it/s]


CPU times: user 11h 9min 44s, sys: 6h 29min 21s, total: 17h 39min 6s
Wall time: 2h 27min 14s


In [ ]:
# save models
model_name = f"bpr_{VER}_{LOCALE}_model_for_inference.npz"
user_ids_name = f"bpr_{VER}_{LOCALE}_user_id2index_for_inference.pickle"
item_ids_name = f"bpr_{VER}_{LOCALE}_item_id2index_for_inference.pickle"
model.save(DIR + "models/task1/" + model_name)
with open(DIR + "models/task1/" + user_ids_name, "wb") as f:
    pickle.dump(user_id2index, f)
with open(DIR + "models/task1/" + item_ids_name, "wb") as f:
    pickle.dump(item_id2index, f)

# save candidates
file_name = f"bpr_{VER}_{LOCALE}_for_inference.parquet"
candidates.to_parquet(DIR + "data/interim/candidates/task1/" + file_name)

In [ ]:
candidates.head()

,session_id,candidate_item,bpr_score,bpr_rank
0,test_phase2_0,B00ZQW91DE,4.037469,1
1,test_phase2_0,B0B3HMH1JP,4.017115,2
2,test_phase2_0,B07T4NH1W4,3.995347,3
3,test_phase2_0,B0B87N98MM,3.992003,4
4,test_phase2_0,B09Y57N2S1,3.982798,5
